In [1]:
import requests

In [2]:
def is_url_image(image_url):
    image_formats = ("image/png", "image/jpeg", "image/jpg")
    r = requests.head(image_url)
    if r.headers["content-type"] in image_formats:
        return True
    return False

In [3]:
url = "https://uploads6.wikiart.org/images/benjamin-west/cupid-and-psyche-1808.jpg!Large.jpg"

In [4]:
is_url_image(url)

False

In [5]:
import sys
sys.path.append('..')

In [6]:
from StyleYourArt import database

Using TensorFlow backend.


In [7]:
db = database.DB()

In [8]:
results = list(db.get_painting_for_style("Baroque", 10))

In [9]:
from StyleYourArt import dataloader
df_paintings = dataloader.load_clean_data('../data/csv', '../data/images/', verbose=True)

... loading painting data from master.csv
... painting data imported from master.csv
... setting dataframe schema
... exporting unique styles to styles.csv
... data loaded
... finding top 25 most popular styles
... filtering data.
   ... record count before filtering: 169677
   ... record count after filtering: 117050
... replace missing values


In [10]:
df_paintings.shape

(117050, 14)

In [12]:
df_paintings['valid_url'] = np.nan

In [13]:
import tqdm

In [24]:
for idx, row in tqdm.tqdm(df_paintings.iterrows()):
    row['valid_url'] = db.is_url_image(row['image'])

117050it [17:52:23,  1.82it/s]


In [27]:
!ls

1_EDA.ipynb         3_Models.ipynb      Untitled.ipynb
2_Bottleneck.ipynb  4_Predictions.ipynb


In [28]:
backup = df_paintings.copy()
backup.to_csv('backup.csv')

In [56]:
df_paintings['valid_url']= df_paintings['valid_url'].astype(np.float16)
df_paintings['valid_url'] = 2.
df_paintings = df_paintings.reset_index()

In [57]:
def is_url_image(image_url):
    image_formats = ("image/png", "image/jpeg", "image/jpg")
    r = requests.head(image_url)
    if r.headers["content-type"] in image_formats:
        return 1.
    return 0.

In [67]:
for idx in tqdm.tqdm(range(df_paintings.shape[0])):
    if df_paintings.loc[idx, 'valid_url']==2:
        df_paintings.loc[idx, 'valid_url'] = is_url_image(df_paintings.loc[idx, 'image'])
    if idx%10000==0:
        df_paintings.to_csv('backup.csv')

100%|██████████| 117050/117050 [16:42:23<00:00,  1.95it/s]  


In [68]:
df_paintings.to_csv('backup.csv')

In [72]:
df_paintings['valid_url'].value_counts()

1.0    91344
0.0    25706
Name: valid_url, dtype: int64

In [69]:
df_paintings['valid_url'].sum() / df_paintings['valid_url'].shape[0]

0.7803844510892781

In [70]:
df_paintings.to_csv('../data/master_clean.csv')

In [73]:
records = df_paintings[df_paintings['valid_url']==1.].to_dict('records')

## clear existing content
db.paintings.delete_many({})

## update database
db.paintings.insert_many(records)
print("... database updated")

... database updated
